<a href="https://colab.research.google.com/github/govindmurthi21/statlearning/blob/main/ragsimple3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain pymongo bs4 openai tiktoken gradio requests lxml unstructured

In [2]:
from google.colab import userdata
from pymongo import MongoClient
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import DirectoryLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base

In [6]:
client = MongoClient(userdata.get("mongo_conn"))
db_name = "langchain_demo"
collection_name = "collection_of_text_blobs"
collection = client[db_name][collection_name]
loader = DirectoryLoader('./sample_data/sample_files', glob='./*.txt', show_progress=True)
data = loader.load()
embeddings = OpenAIEmbeddings(openai_api_key=userdata.get("openapi_key"))
vector_store = MongoDBAtlasVectorSearch.from_documents(data, embeddings, collection=collection)

100%|██████████| 3/3 [00:00<00:00, 24.29it/s]


In [6]:
client = MongoClient(userdata.get("mongo_conn"))
db_name = "langchain_demo"
collection_name = "collection_of_text_blobs"
collection = client[db_name][collection_name]
print(collection.count_documents({}))
embeddingsNew = OpenAIEmbeddings(openai_api_key=userdata.get("openapi_key"))
vectorStore = MongoDBAtlasVectorSearch(collection, embeddingsNew, index_name="vector_index")

def query_data(query):
  print(query)
  docs = vectorStore.similarity_search(query, K=1)
  print(docs)
  if docs and len(docs) > 0:
    as_output = docs[0].page_content
  else:
    as_output = None
  llm = OpenAI(openai_api_key=userdata.get("openapi_key"), temperature=0)
  retriever = vectorStore.as_retriever()
  qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
  retriever_output = qa.run(query)
  return as_output, retriever_output

with gr.Blocks(theme=Base(), title="Question Answering App using Vector Search + RAG") as demo:
    gr.Markdown(
        """
        # Question Answering App using Atlas Vector Search + RAG Architecture
        """)
    textbox = gr.Textbox(label="Enter your Question:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (returns text field as is):")
        output2 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQA + OpenAI LLM:")

# Call query_data function upon clicking the Submit button

    button.click(query_data, textbox, outputs=[output1, output2])

demo.launch(debug=True)

3
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://408e089cf4ca561346.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


what did alfred ask bruce? 
[Document(page_content="Alfred: Hi, can you explain to me how compression works in MongoDB? Bruce: Sure! MongoDB supports compression of data at rest. It uses either zlib or snappy compression algorithms at the collection level. When data is written, MongoDB compresses and stores it compressed. When data is read, MongoDB uncompresses it before returning it. Compression reduces storage space requirements. Alfred: Interesting, that's helpful to know. Can you also tell me how indexes are stored in MongoDB? Bruce: MongoDB indexes are stored in B-trees. The internal nodes of the B-trees contain keys that point to children nodes or leaf nodes. The leaf nodes contain references to the actual documents stored in the collection. Indexes are stored in memory and also written to disk. The in-memory B-trees provide fast access for queries using the index.Alfred: Ok that makes sense. Does MongoDB compress the indexes as well?Bruce: Yes, MongoDB also compresses the index 